In [68]:
import random as rd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error #, r2_score
from sklearn.model_selection import train_test_split

import gspread
from sqlalchemy import create_engine
import re
from main.apps.etl.models import GoogleSheetReport

gc = gspread.service_account(filename="./credentials.json")

import environ
env = environ.Env()
database_url = env.str(
    var="DATABASE_URL", default="postgres://postgres:@postgres:5432/postgres"
)
engine = create_engine(database_url, echo=False)

In [69]:
sales = pd.read_sql("etl_finance2021", engine)
sales

,Invoice Date,Incoice No.,Billing Type,Group,Customer Name,PO/QT No.,Month,Due date,Amount befor vat,Vat,Total,Status,Receive date,Tax Invoice No.,Payment Detail
0,13-Jan-2021,IV202101001,Monthly,Etax : BNG,HomePro,,2,31-Jan-2021,"12,386.72",867.07,"13,253.79",Paid,17-Feb-2021,RE202102026,Bank Tranfer
1,13-Jan-2021,IV202101002,Monthly,Etax : BNG,Thai Samsung,QT202005001,2,31-Jan-2021,"3,533.04",247.31,"3,780.35",Paid,17-Feb-2021,RE202102027,Bank Tranfer
2,13-Jan-2021,IV202101003,Monthly,Etax : BNG,Dplus,QT202010009,2,31-Jan-2021,"30,507.72","2,135.54","32,643.26",Paid,17-Feb-2021,RE202102028,Bank Tranfer
3,13-Jan-2021,IV202101004,Monthly,Etax : BNG,Digital Venture,PO2019-5,2,31-Jan-2021,"2,250.00",157.50,"2,407.50",Paid,17-Feb-2021,RE202102029,Bank Tranfer
4,,IV202101005,Monthly,SCB : BCM,10X / Honda,PO2019-5,12,31-Jan-2021,"20,426.01","1,429.82","21,855.83",void,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,8-Mar-2021,IV202103035,Monthly,SCB : BCM,10X / Honda,PO2019-5,12,2-Apr-2021,"18,626.01","1,303.82","19,929.83",On Due,,,
92,8-Mar-2021,IV202103036,Change Request,Memo,Chodthanawat,QT202101020,12,5-Apr-2021,"33,000.00","2,310.00","35,310.00",On Due,,,
93,11-Mar-2021,IV202103037,Installment,Project : CDM,Meticuly,P21020001,12,2-Apr-2021,"560,747.66","39,252.34","600,000.00",On Due,,,
94,15-Mar-2021,IV202103038,Monthly,Memo,ส.เขมราฐ,QT202005014,12,15-Apr-2021,"83,600.00","5,852.00","89,452.00",On Due,,,


In [70]:
sales.dtypes
for i,row in sales.iterrows():
    if row['Amount befor vat']:
        sales['Amount befor vat'].iloc[int(i)] = sales['Amount befor vat'].iloc[int(i)].replace(',','')
sales['Amount befor vat'] = sales['Amount befor vat'].astype(float)

In [71]:
# sales = sales[(sales['Amount befor vat'] != "") & (sales['Status '] == 'Paid')]
sales

,Invoice Date,Incoice No.,Billing Type,Group,Customer Name,PO/QT No.,Month,Due date,Amount befor vat,Vat,Total,Status,Receive date,Tax Invoice No.,Payment Detail
0,13-Jan-2021,IV202101001,Monthly,Etax : BNG,HomePro,,2,31-Jan-2021,12386.72,867.07,"13,253.79",Paid,17-Feb-2021,RE202102026,Bank Tranfer
1,13-Jan-2021,IV202101002,Monthly,Etax : BNG,Thai Samsung,QT202005001,2,31-Jan-2021,3533.04,247.31,"3,780.35",Paid,17-Feb-2021,RE202102027,Bank Tranfer
2,13-Jan-2021,IV202101003,Monthly,Etax : BNG,Dplus,QT202010009,2,31-Jan-2021,30507.72,"2,135.54","32,643.26",Paid,17-Feb-2021,RE202102028,Bank Tranfer
3,13-Jan-2021,IV202101004,Monthly,Etax : BNG,Digital Venture,PO2019-5,2,31-Jan-2021,2250.00,157.50,"2,407.50",Paid,17-Feb-2021,RE202102029,Bank Tranfer
4,,IV202101005,Monthly,SCB : BCM,10X / Honda,PO2019-5,12,31-Jan-2021,20426.01,"1,429.82","21,855.83",void,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,8-Mar-2021,IV202103035,Monthly,SCB : BCM,10X / Honda,PO2019-5,12,2-Apr-2021,18626.01,"1,303.82","19,929.83",On Due,,,
92,8-Mar-2021,IV202103036,Change Request,Memo,Chodthanawat,QT202101020,12,5-Apr-2021,33000.00,"2,310.00","35,310.00",On Due,,,
93,11-Mar-2021,IV202103037,Installment,Project : CDM,Meticuly,P21020001,12,2-Apr-2021,560747.66,"39,252.34","600,000.00",On Due,,,
94,15-Mar-2021,IV202103038,Monthly,Memo,ส.เขมราฐ,QT202005014,12,15-Apr-2021,83600.00,"5,852.00","89,452.00",On Due,,,


In [72]:
sales.sum()

Invoice Date        13-Jan-202113-Jan-202113-Jan-202113-Jan-202113...
Incoice No.         IV202101001IV202101002IV202101003IV202101004IV...
Billing Type        MonthlyMonthlyMonthlyMonthlyMonthlyMonthlyMont...
Group               Etax : BNGEtax : BNGEtax : BNGEtax : BNGSCB : ...
Customer Name       HomeProThai Samsung DplusDigital Venture 10X /...
PO/QT No.           QT202005001QT202010009PO2019-5PO2019-5PO2019-1...
Month               2222122222222221233211211211111333333333312121...
Due date            31-Jan-202131-Jan-202131-Jan-202131-Jan-202131...
Amount befor vat                                          2.69804e+06
Vat                 867.07247.312,135.54157.501,429.82189.0094.504...
Total               13,253.793,780.3532,643.262,407.5021,855.832,8...
Status              PaidPaidPaidPaidvoidPaidPaidPaidPaidPaidPaidPa...
Receive date        17-Feb-202117-Feb-202117-Feb-202117-Feb-202117...
Tax Invoice No.     RE202102026RE202102027RE202102028RE202102029RE...
Payment Detail      

In [32]:
def job_all_sale():
    data = GoogleSheetReport.objects.filter(type="sale")
    for item in data:
        url = str(item.url)
        year = item.year
        finance_job(url, year)

In [41]:
def finance_job(_url, y=None):
    url = str(_url)

    year = y or datetime.datetime.now().year

    workbook = gc.open_by_url(url)

    list_data = list()

    for i in range(2):
        sh = workbook.get_worksheet(i + 2)

        if year == 2020:
            sheet = sh.get_all_values()[1:]
        else:
            sheet = sh.get_all_values()

        dataframe = pd.DataFrame(sheet)

        dataframe = dataframe.T.set_index(0).T

        list_column_name = dataframe.columns.to_list()

        col_select = list_column_name[:15]  # 15
        data_select_by_col = dataframe[col_select]
        print(data_select_by_col)
        if year == 2020:
            data = data_select_by_col.drop(
                data_select_by_col[(data_select_by_col[col_select[2]] == "")].index
            )
        else:
            data = data_select_by_col.drop(
                data_select_by_col[(data_select_by_col[col_select[1]] == "")].index
            )

        list_data.append(data)

    df = pd.concat(list_data, axis=0)

In [42]:
job_all_sale()

0  Invoice Date  Incoice No. Billing Type       Group  \
1   13-Jan-2021  IV202101001      Monthly  Etax : BNG   
2   13-Jan-2021  IV202101002      Monthly  Etax : BNG   
3   13-Jan-2021  IV202101003      Monthly  Etax : BNG   
4   13-Jan-2021  IV202101004      Monthly  Etax : BNG   
5                IV202101005      Monthly   SCB : BCM   
6   13-Jan-2021  IV202101006      Monthly  Etax : BNG   
7   13-Jan-2021  IV202101007      Monthly  Etax : BNG   
8   13-Jan-2021  IV202101008      Monthly  Etax : BNG   
9   13-Jan-2021  IV202101009      Monthly  Etax : BNG   
10  13-Jan-2021  IV202101010      Monthly  Etax : BNG   
11   5-Jan-2021  IV202101011      Monthly  Etax : CHM   
12   5-Jan-2021  IV202101012      Monthly  Etax : CHM   
13   5-Jan-2021  IV202101013      Monthly  Etax : CHM   
14   5-Jan-2021  IV202101014      Monthly  Etax : CHM   
15   5-Jan-2021  IV202101015      Monthly        Memo   
16   5-Jan-2021  IV202101016      Monthly  Etax : CHM   
17   5-Jan-2021  IV202101017   

In [77]:
def week(dd):
    sw = {
        "1": "Monday",
        "2": "Tuesday",
        "3": "Wednesday",
        "4": "Thursday",
        "5": "Friday",
        "6": "Saturday",
        "7": "Sunday"
    }
    return sw.get(dd, "Please type only number (1-7)")


day = input("type num day: ")
day_name = week(day)
print(day_name)

type num day: 12
Please type only number (1-7)


In [78]:
command = input()
match command.split():
    case ["quit"]:
        quit()
    case ["load", filename]:
        load_from(filename)
    case ["save", filename]:
        save_to(filename)
    case _:
        print (f"Command '{command}' not understood")

SyntaxError: invalid syntax (<ipython-input-78-3e9ff01870b0>, line 2)